
Определить в файле ABVG текущую стадии по следующим условиям:

1.1. Разметка стадий движется от начала исторических данных к текущему времени. В файл добавляются столбцы соответствующей стадии и булевым значением в каждой строке файла, указывающим на выполнение условия стадии для конкретной записи. Таким образом каждая запись файла должна иметь значение ЛОЖЬ/ИСТИНА в столбце каждой стадии. 

1.2. Должны соблюдаться ответвления стадий, например если найдена стадия 1 и ожидается стадия 2, то формируется средний объем стадии 1 и параллельно идет учет данных стадии 0 и может снова начать зарождаться стадия 1, таким образом может быть ситуация, когда уже сформированная стадия 1 прерывается вновь созданной стадией 1. Все стадии, триггеры и цели длятся до тех пор, пока выполняется их условие.

1.3. Стадия 0.  Ежедневный объем колеблется от 0 до х10 от среднего объема за 100 дней, при этом цена не растет. 

"100 - дней” блок, формируемый построчно, т.е. берем первую строку и ее значение записываем в среднее. Далее вторую строку и пересчитываем среднюю и переходим к следующей строке. Алгоритм работает пока не собрали 100 строк. Получаем сдвигающийся после достижения отметки 100 записей блок (т.е. мы должны смещать его всегда на +1 после появления доступной новой записи) например начиная с 5 записи блок будет  размером 100 записей следовательно с 5 -105, после появления или наличия доступной новой записи 6-106 , 7 - 107 и т.д. Это справедливо для всех правил с диапазоном дат (100 дней, декада и т.д.). 

1.4. Стадия 1. Рост дневных объемов в течении нескольких дней (не менее 3 дней за декаду) на >x5 от среднего значения объема за 100 дней, при этом цена закрытия не превышает открытие на не более чем на 25% для каждого дня с указанным объемом. Постоянно считаем средний объем данной стадии.

1.5. Стадия 2. Дневной объем >х5-50 от среднего объема стадии 1. При этом цена закрытия <= цены открытия этого дня. Постоянно считаем среднюю МАКСимальной цены данной стадии, а также средний объем данной стадии.




In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#data = pd.read_excel('drive/MyDrive/stages_v5_4.xlsx', index_col='Unnamed: 0')
data = pd.read_excel('drive/MyDrive/ABVG.xlsx')

In [ ]:
data

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий
0,2021-05-03,68303969.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN
1,2021-04-30,71543631.0,0.0003,0.0004,0.0003,0.0003,0.0003,NaN
2,2021-04-29,68309894.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN
3,2021-04-28,128170551.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN
4,2021-04-27,313368602.0,0.0004,0.0004,0.0003,0.0004,0.0004,NaN
...,...,...,...,...,...,...,...,...
4998,2001-06-19,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN
4999,2001-06-18,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN
5000,2001-06-15,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN
5001,2001-06-14,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         5003 non-null   datetime64[ns]
 1   Volume       4658 non-null   float64       
 2   Open         4658 non-null   float64       
 3   High         4658 non-null   float64       
 4   Low          4658 non-null   float64       
 5   Close        4658 non-null   float64       
 6   Adj Close    4658 non-null   float64       
 7   Комментарий  3 non-null      object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 312.8+ KB


In [ ]:
#filter missing values
data = data[~data[['Date','Volume','Open','High','Low','Close', 'Adj Close']].isnull().any(axis=1)]
data = data.reset_index(drop=True)

In [ ]:
def compute_window_mean(data_frame, window_size):
  return data_frame['Volume'].rolling(window=window_size).mean().shift(-window_size+1)

In [ ]:
#add Mean_100_days column
data = data.assign(Mean_100_days = pd.Series(compute_window_mean(data, 100)).values)

In [ ]:
data

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days
0,2021-05-03,68303969.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.021904e+08
1,2021-04-30,71543631.0,0.0003,0.0004,0.0003,0.0003,0.0003,NaN,2.015074e+08
2,2021-04-29,68309894.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.011065e+08
3,2021-04-28,128170551.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.004234e+08
4,2021-04-27,313368602.0,0.0004,0.0004,0.0003,0.0004,0.0004,NaN,1.992147e+08
...,...,...,...,...,...,...,...,...,...
4653,2001-06-19,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN
4654,2001-06-18,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN
4655,2001-06-15,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN
4656,2001-06-14,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN


In [ ]:
def stage0(x):
  return (0.0 <= x['Volume'] < x['Mean_100_days']*10.0 and x['Open'] >= x['Close'])

In [ ]:
data.apply(stage0, axis=1)

0        True
1        True
2        True
3        True
4        True
        ...  
4653    False
4654    False
4655    False
4656    False
4657    False
Length: 4658, dtype: bool

In [ ]:
#add stage0 column
data = data.assign(stage0 = data.apply(stage0, axis=1).values)

**Выведем строки, помеченные в файле как стадия0**

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data[data.index.isin(list(range(105,237)))][['Date','stage0', 'Volume', 'Mean_100_days', 'Open', 'Close']])

          Date  stage0      Volume  Mean_100_days    Open   Close
105 2020-11-30    True    510000.0      440953.89  0.0001  0.0001
106 2020-11-27    True      3000.0      463853.88  0.0001  0.0001
107 2020-11-25    True         0.0      463823.88  0.0001  0.0001
108 2020-11-24    True   3101249.0      463823.88  0.0001  0.0001
109 2020-11-23    True         0.0      432811.39  0.0001  0.0001
110 2020-11-20    True   1400000.0      432811.39  0.0001  0.0001
111 2020-11-19    True    500000.0      418811.39  0.0001  0.0001
112 2020-11-18    True    200000.0      413811.39  0.0001  0.0001
113 2020-11-17    True         0.0      411811.39  0.0001  0.0001
114 2020-11-16    True    103000.0      411811.39  0.0001  0.0001
115 2020-11-13    True   1501000.0      410781.39  0.0001  0.0001
116 2020-11-12    True      1000.0      395771.39  0.0001  0.0001
117 2020-11-11    True      2000.0      395761.39  0.0001  0.0001
118 2020-11-10    True      3000.0      395741.39  0.0001  0.0001
119 2020-1

В некоторых строках условие не выполняется. Однако средний объем считается верно, например для строки 130:


In [ ]:
data[data.index.isin(list(range(130,230)))]['Volume'].mean() == data.iloc[130]['Mean_100_days']

True

Цена же во всех строках не меняется. Поэтому считаем, что stage0 вычисляется корректно.

**Выясним, в каких строках выполняется условие для стадии1**

In [ ]:
def stage1_trigger(x):
    return (x['Volume'] > x['Mean_100_days']*5.0) and (x['Close'] <= x['Open']*1.25)

In [ ]:
data = data.assign(Stage1_trigger = data.apply(stage1_trigger, axis=1).values)

In [ ]:
data[data['Stage1_trigger']]

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger
54,2021-02-12,776562877.0,0.001700,0.0017,0.0011,0.001500,0.001500,NaN,1.291670e+08,True,True
64,2021-01-29,838998637.0,0.000400,0.0005,0.0003,0.000400,0.000400,NaN,9.843543e+07,True,True
65,2021-01-28,502162993.0,0.000400,0.0005,0.0003,0.000350,0.000350,NaN,9.004544e+07,True,True
68,2021-01-25,446213334.0,0.000600,0.0008,0.0004,0.000700,0.000700,NaN,7.837623e+07,False,True
71,2021-01-20,845214122.0,0.000200,0.0003,0.0001,0.000200,0.000200,NaN,7.120090e+07,False,True
...,...,...,...,...,...,...,...,...,...,...,...
4499,2002-02-11,14800.0,33.000000,34.0000,33.0000,34.000000,34.000000,NaN,2.731500e+03,False,True
4500,2002-02-08,14800.0,31.799999,33.0000,31.5000,32.400002,32.400002,NaN,2.583500e+03,False,True
4506,2002-01-31,21750.0,29.000000,30.5000,28.5000,30.500000,30.500000,NaN,2.318900e+03,False,True
4511,2002-01-24,21020.0,24.000000,26.0000,24.0000,26.000000,26.000000,NaN,2.050000e+03,False,True


In [ ]:
def compute_stage1(data_frame, window_size):
    return data_frame['Stage1_trigger'].rolling(window=window_size).apply(stage1_window, raw=False).shift(-window_size+1)

In [ ]:
def stage1_window(x):
    #print(x[x > 0].count() >= 3)
    return x[x > 0].count() >= 3

In [ ]:
stage1_series = pd.Series(compute_stage1(data, 10))
stage1_series[stage1_series.isna()] = 0.0

In [ ]:
data = data.assign(stage1 = stage1_series.values.astype('bool'))

In [ ]:
data[data.index.isin(list(range(98,110)))][['Date', 'Volume', 'Mean_100_days','Close','Open','Stage1_trigger','stage1']]

,Date,Volume,Mean_100_days,Close,Open,Stage1_trigger,stage1
98,2020-12-09,0.0,1434885.87,0.00010,0.00010,False,True
99,2020-12-08,35259997.0,1434885.87,0.00010,0.00010,True,True
100,2020-12-07,0.0,1082285.90,0.00005,0.00005,False,True
101,2020-12-04,31455001.0,1167583.90,0.00005,0.00010,True,True
102,2020-12-03,0.0,853033.89,0.00010,0.00010,False,True
103,2020-12-02,7300000.0,853033.89,0.00010,0.00010,True,True
104,2020-12-01,33908000.0,780033.89,0.00010,0.00010,True,False
105,2020-11-30,510000.0,440953.89,0.00010,0.00010,False,False
106,2020-11-27,3000.0,463853.88,0.00010,0.00010,False,False
107,2020-11-25,0.0,463823.88,0.00010,0.00010,False,False


**Пронумеруем найденные stage1**

In [ ]:
def stage1_count_window(x, info):
    res = -1
    if x.iloc[0]:
        res = info['num']

    if x.diff().any() and x.iloc[0]:
        info['num'] += 1

    return res

In [ ]:
info = {}
item = data.iloc[data.index.min()]
info['isStage1'] = item['stage1']
info['num'] = 0

In [ ]:
info

{'isStage1': False, 'num': 0}

In [ ]:
#data['stage1'].rolling(window=2).apply(stage1_count_window, args=(info,)).shift(-1)

In [ ]:
data = data.assign(stage1_num = pd.Series(data['stage1'].rolling(window=2).apply(stage1_count_window, args=(info,)).shift(-1)).values.astype('int'))

In [ ]:
data

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger,stage1,stage1_num
0,2021-05-03,68303969.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.021904e+08,True,False,False,-1
1,2021-04-30,71543631.0,0.0003,0.0004,0.0003,0.0003,0.0003,NaN,2.015074e+08,True,False,False,-1
2,2021-04-29,68309894.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.011065e+08,True,False,False,-1
3,2021-04-28,128170551.0,0.0004,0.0004,0.0003,0.0003,0.0003,NaN,2.004234e+08,True,False,False,-1
4,2021-04-27,313368602.0,0.0004,0.0004,0.0003,0.0004,0.0004,NaN,1.992147e+08,True,False,False,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4653,2001-06-19,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN,False,False,False,-1
4654,2001-06-18,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN,False,False,False,-1
4655,2001-06-15,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN,False,False,False,-1
4656,2001-06-14,0.0,3.3300,3.3300,3.3300,3.3300,3.3300,NaN,NaN,False,False,False,-1


In [ ]:
data[['stage1','stage1_num']][data.index.isin(list(range(88,110)))]

,stage1,stage1_num
88,True,1
89,True,1
90,True,1
91,True,1
92,True,1
93,True,1
94,True,1
95,True,1
96,True,1
97,True,1


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(data[data.index.isin(list(range(100,200)))][['stage1','stage1_num']])

     stage1  stage1_num
100    True           1
101    True           1
102    True           1
103    True           1
104   False          -1
105   False          -1
106   False          -1
107   False          -1
108   False          -1
109   False          -1
110   False          -1
111   False          -1
112   False          -1
113   False          -1
114   False          -1
115   False          -1
116   False          -1
117   False          -1
118   False          -1
119   False          -1
120   False          -1
121   False          -1
122   False          -1
123   False          -1
124   False          -1
125   False          -1
126   False          -1
127    True           2
128    True           2
129    True           2
130    True           2
131   False          -1
132   False          -1
133   False          -1
134   False          -1
135   False          -1
136   False          -1
137   False          -1
138   False          -1
139   False          -1
140   False     

**для каждого найденного stage1 найдем скользящее среднее(отдельно для каждого номера)**

In [ ]:
tmp = data[['stage1_num','Volume']][::-1].groupby('stage1_num').expanding(on='Volume').mean()

In [ ]:
tmp.index.get_level_values(1)


Int64Index([4657, 4656, 4655, 4654, 4653, 4652, 4651, 4650, 4649, 4648,
            ...
            3742, 3750, 3749, 4494, 4493, 4492, 4491, 4499, 4498, 4497],
           dtype='int64', length=4658)

In [ ]:
tmp['Volume'].values

array([ 1800.        ,     0.        ,     0.        , ...,
       14800.        ,  7475.        ,  5933.33333333])

In [ ]:
data['mean_stage1'] = pd.DataFrame(tmp['Volume'].values, index = tmp.index.get_level_values(1)).sort_index()

In [ ]:
#data[['Date','Volume','stage1_num','mean_stage1']][data.index.isin(list(range(78,110)))]

Очищаем значения, у которых нет stage1

In [ ]:
data.loc[data['stage1_num'] < 0, 'stage1_num'] = -1

In [ ]:
data.loc[data['stage1_num'] < 0, 'mean_stage1'] = -1

**Вычислим stage2**

In [ ]:
def stage2(x):
    if x['mean_stage1'] < 0:
      return False
    return x['mean_stage1']*5.0 < x['Volume'] < x['mean_stage1']*50.0 and x['Close'] <= x['Open']


In [ ]:
data['stage2'] = data.apply(stage2, axis=1)

In [ ]:
data[data['stage2']]

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger,stage1,stage1_num,mean_stage1,stage2
95,2020-12-14,979854529.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,13465026.05,False,True,True,1,1.419032e+08,True
96,2020-12-11,223257694.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,3667480.76,False,True,True,1,3.715931e+07,True


**Нумеруем найденные stage2**

In [ ]:
def stage2_count_window(x, info):
    res = -1
    if x.iloc[0]:
        res = info['num']

    if x.diff().any() and x.iloc[0]:
        info['num'] += 1

    return res

In [ ]:
info = {}
item = data.iloc[data.index.min()]
info['isStage2'] = item['stage1']
info['num'] = 0

In [ ]:
info

{'isStage2': False, 'num': 0}

In [ ]:
data = data.assign(stage2_num = pd.Series(data['stage2'].rolling(window=2).apply(stage2_count_window, args=(info,)).shift(-1)).values.astype('int'))

In [ ]:
data[data['stage2_num'] == 0]

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger,stage1,stage1_num,mean_stage1,stage2,stage2_num
95,2020-12-14,979854529.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,13465026.05,False,True,True,1,1.419032e+08,True,0
96,2020-12-11,223257694.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,3667480.76,False,True,True,1,3.715931e+07,True,0


**Вычисляем скользящее среднее объемов и максимальной цены в группах**

In [ ]:
tmp = data[['stage2_num','Volume']][::-1].groupby('stage2_num').expanding(on='Volume').mean()

In [ ]:
tmp.index.get_level_values(1)


Int64Index([4657, 4656, 4655, 4654, 4653, 4652, 4651, 4650, 4649, 4648,
            ...
               7,    6,    5,    4,    3,    2,    1,    0,   96,   95],
           dtype='int64', length=4658)

In [ ]:
tmp['Volume'].values

array([1.80000000e+03, 0.00000000e+00, 0.00000000e+00, ...,
       1.37480955e+07, 2.23257694e+08, 6.01556112e+08])

In [ ]:
data['mean_stage2'] = pd.DataFrame(tmp['Volume'].values, index = tmp.index.get_level_values(1)).sort_index()

In [ ]:
data.loc[data['stage2_num'] < 0, 'mean_stage2'] = -1

In [ ]:
data[data['stage2']]

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger,stage1,stage1_num,mean_stage1,stage2,stage2_num,mean_stage2
95,2020-12-14,979854529.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,13465026.05,False,True,True,1,1.419032e+08,True,0,601556111.5
96,2020-12-11,223257694.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,3667480.76,False,True,True,1,3.715931e+07,True,0,223257694.0


In [ ]:
tmp = data[['stage2_num','High']][::-1].groupby('stage2_num').expanding(on='High').mean()

In [ ]:
tmp.index.get_level_values(1)


Int64Index([4657, 4656, 4655, 4654, 4653, 4652, 4651, 4650, 4649, 4648,
            ...
               7,    6,    5,    4,    3,    2,    1,    0,   96,   95],
           dtype='int64', length=4658)

In [ ]:
tmp['High'].values

array([3.33000000e+00, 3.33000000e+00, 3.33000000e+00, ...,
       3.72275795e+00, 2.00000000e-04, 2.00000000e-04])

In [ ]:
data['mean_High_stage2'] = pd.DataFrame(tmp['High'].values, index = tmp.index.get_level_values(1)).sort_index()

In [ ]:
data.loc[data['stage2_num'] < 0, 'mean_High_stage2'] = -1

In [ ]:
data[data['stage2']]

,Date,Volume,Open,High,Low,Close,Adj Close,Комментарий,Mean_100_days,stage0,Stage1_trigger,stage1,stage1_num,mean_stage1,stage2,stage2_num,mean_stage2,mean_High_stage2
95,2020-12-14,979854529.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,13465026.05,False,True,True,1,1.419032e+08,True,0,601556111.5,0.0002
96,2020-12-11,223257694.0,0.0002,0.0002,0.0001,0.0001,0.0001,NaN,3667480.76,False,True,True,1,3.715931e+07,True,0,223257694.0,0.0002


Записываем результат в файл


In [ ]:
data.to_excel('drive/MyDrive/result.xlsx')